<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>
Author: Yury Kashnitsky, Data Scientist at Mail.Ru Group

This material is subject to the terms and conditions of the license [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Free use is permitted for any non-comercial purpose with an obligatory indication of the names of the authors and of the source.

## <center>Assignment #6. Part 1
### <center> Beating benchmarks in "Catch Me If You Can: Intruder Detection through Webpage Session Tracking"
    
[Competition](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). The task is to beat "Assignment 6 baseline".

In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

Reading original data

In [3]:
train_df = pd.read_csv(r"train_sessions.csv", index_col='session_id')
test_df = pd.read_csv(r"test_sessions.csv", index_col='session_id')

In [4]:
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [ ]:
# проверка на даты
train_df = train_df.sort_values(by='time1')

train_df.head()

In [5]:
train_df[times]

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
21669,2013-01-12 08:05:57,2013-01-12 08:05:57,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
54843,2013-01-12 08:37:23,2013-01-12 08:37:23,2013-01-12 09:07:07,2013-01-12 09:07:09,NaT,NaT,NaT,NaT,NaT,NaT
77292,2013-01-12 08:50:13,2013-01-12 08:50:14,2013-01-12 08:50:15,2013-01-12 08:50:15,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:17,2013-01-12 08:50:17
114021,2013-01-12 08:50:17,2013-01-12 08:50:17,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:20
146670,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:22,2013-01-12 08:50:22,2013-01-12 08:50:22
242171,2013-01-12 08:50:22,2013-01-12 08:50:23,2013-01-12 08:50:23,2013-01-12 08:50:23,2013-01-12 08:50:24,2013-01-12 08:50:24,2013-01-12 08:50:24,2013-01-12 08:50:24,2013-01-12 08:50:25,2013-01-12 08:50:25
57157,2013-01-12 08:50:25,2013-01-12 08:50:26,2013-01-12 08:50:26,2013-01-12 08:50:26,2013-01-12 08:50:26,2013-01-12 08:50:27,2013-01-12 08:50:27,2013-01-12 08:50:27,2013-01-12 08:50:28,2013-01-12 08:50:28
240201,2013-01-12 08:50:28,2013-01-12 08:50:28,2013-01-12 08:50:28,2013-01-12 08:50:29,2013-01-12 08:50:29,2013-01-12 08:50:29,2013-01-12 08:50:30,2013-01-12 08:50:30,2013-01-12 08:50:31,2013-01-12 08:50:31
210686,2013-01-12 08:50:31,2013-01-12 08:50:32,2013-01-12 08:50:32,2013-01-12 08:50:33,2013-01-12 08:50:33,2013-01-12 08:50:33,2013-01-12 08:50:34,2013-01-12 08:50:35,2013-01-12 08:50:36,2013-01-12 08:50:36


In [6]:
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

In [7]:
y_train = train_df['target']

full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# Index to split the training and test data sets
idx_split = train_df.shape[0]


Separate target feature 

In [8]:
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [9]:
full_sites['str'] = full_sites.apply(lambda row: ' '.join(str(x) for x in row.values.tolist()), axis=1)

full_sites['str'].head()

session_id
21669                       56 55 0 0 0 0 0 0 0 0
54843                     56 55 56 55 0 0 0 0 0 0
77292     946 946 951 946 946 945 948 784 949 946
114021    945 948 949 948 945 946 947 945 946 946
146670    947 950 948 947 950 952 946 951 946 947
Name: str, dtype: object

In [10]:
vectorizer = TfidfVectorizer(ngram_range=(1,10), max_features= 100000)
out = vectorizer.fit_transform(full_sites['str'])
out.shape

(336358, 100000)

In [ ]:
def delta_mins(date1, date2):
      return int((date2-date1).total_seconds())

Build Tf-Idf features based on sites. You can use `ngram_range`=(1, 3) and `max_features`=100000 or more

In [13]:
times = ['time%s' % i for i in range(1,11)]
full_time = pd.DataFrame()
full_time['duration'] = full_df[times].apply(lambda row: delta_mins(row.min(), row.max()), axis=1)

In [14]:
full_times_only  = pd.DataFrame(index = full_df.index)
full_times_only = full_df[times]

In [86]:
full_times_only.head()

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
21669,2013-01-12 08:05:57,2013-01-12 08:05:57,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
54843,2013-01-12 08:37:23,2013-01-12 08:37:23,2013-01-12 09:07:07,2013-01-12 09:07:09,NaT,NaT,NaT,NaT,NaT,NaT
77292,2013-01-12 08:50:13,2013-01-12 08:50:14,2013-01-12 08:50:15,2013-01-12 08:50:15,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:16,2013-01-12 08:50:17,2013-01-12 08:50:17
114021,2013-01-12 08:50:17,2013-01-12 08:50:17,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:18,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:19,2013-01-12 08:50:20
146670,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:20,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:21,2013-01-12 08:50:22,2013-01-12 08:50:22,2013-01-12 08:50:22


In [15]:
time_features = pd.DataFrame(index=full_df.index)
  

In [16]:
day_features = pd.DataFrame(index=full_df.index)

In [17]:
days = ['day%s' % i for i in range(1,11)]

In [18]:
day_features['day1'] = full_df['time1'].apply(lambda day: day.weekday())
day_features['day2'] = full_df['time2'].apply(lambda day: day.weekday())
day_features['day3'] = full_df['time3'].apply(lambda day: day.weekday())
day_features['day4'] = full_df['time4'].apply(lambda day: day.weekday())
day_features['day5'] = full_df['time5'].apply(lambda day: day.weekday())
day_features['day6'] = full_df['time6'].apply(lambda day: day.weekday())
day_features['day7'] = full_df['time7'].apply(lambda day: day.weekday())
day_features['day8'] = full_df['time8'].apply(lambda day: day.weekday())
day_features['day9'] = full_df['time9'].apply(lambda day: day.weekday())
day_features['day10'] = full_df['time10'].apply(lambda day: day.weekday())

In [19]:
hours = ['hour%s' % i for i in range(1,11)]

In [20]:
time_features['hour1'] = full_df['time1'].apply(lambda day: day.hour)
time_features['hour2'] = full_df['time2'].apply(lambda day: day.hour)
time_features['hour3'] = full_df['time3'].apply(lambda day: day.hour)
time_features['hour4'] = full_df['time4'].apply(lambda day: day.hour)
time_features['hour5'] = full_df['time5'].apply(lambda day: day.hour)
time_features['hour6'] = full_df['time6'].apply(lambda day: day.hour)
time_features['hour7'] = full_df['time7'].apply(lambda day: day.hour)
time_features['hour8'] = full_df['time8'].apply(lambda day: day.hour)
time_features['hour9'] = full_df['time9'].apply(lambda day: day.hour)
time_features['hour10'] = full_df['time10'].apply(lambda day: day.hour)

In [21]:
time_features.head()

,hour1,hour2,hour3,hour4,hour5,hour6,hour7,hour8,hour9,hour10
session_id,,,,,,,,,,
21669,8,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54843,8,8.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
77292,8,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
114021,8,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
146670,8,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0


In [22]:
time_features = pd.get_dummies(time_features,columns=hours, sparse = False)

In [23]:
day_features = pd.get_dummies(day_features,columns=days, sparse = False)

In [123]:
time_features.head()

,hour1_7,hour1_8,hour1_9,hour1_10,hour1_11,hour1_12,hour1_13,hour1_14,hour1_15,hour1_16,...,hour10_14.0,hour10_15.0,hour10_16.0,hour10_17.0,hour10_18.0,hour10_19.0,hour10_20.0,hour10_21.0,hour10_22.0,hour10_23.0
session_id,,,,,,,,,,,,,,,,,,,,,
21669,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54843,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77292,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114021,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146670,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
full_time['start_hour'] = full_df['time1'].apply(lambda ts: ts.hour)

In [30]:
full_time['weekday'] = full_df['time1'].apply(lambda ts:ts.weekday())

In [31]:
full_time['morning'] = full_time['start_hour'].apply(lambda i:1 if i <=11 else 0)

In [32]:
full_time['work_time'] = full_time['start_hour'].apply(lambda i:1 if i >=9 and i<=16 else 0)

In [47]:
from scipy.sparse import hstack
matrix = hstack([out, time_features, day_features, full_time]).tocsr(copy=False)

In [25]:
def get_auc_lr_valid(X, y, C=1.8, seed=17, ratio = 0.9):
    # Split the data into the training and validation sets
    idx = int(round(X.shape[0] * ratio))
    # Classifier training
    lr = LogisticRegression(C=C, random_state=seed, n_jobs=-1).fit(X[:idx, :], y[:idx])
    # Prediction for validation set
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # Calculate the quality
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [48]:
%%time
# Select the training set from the united dataframe (where we have the answers)
X_train = matrix[:idx_split, :]

# Calculate metric on the validation set
print(get_auc_lr_valid(X_train, y_train))

0.980674904231217
Wall time: 2min 12s


In [49]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [50]:
lr = LogisticRegression(C=1.8, random_state=17).fit(X_train, y_train)

# Make a prediction for test data set
X_test = matrix[idx_split:,:]
y_test = lr.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted
write_to_submission_file(y_test, 'baseline_1.csv')